# Twitter Test

## Tweepy

How to get up-to-date user tweets?

In [6]:
# proxy
from fp.fp import FreeProxy  # https://github.com/jundymek/free-proxy

def gen_proxy_list(timeout=5, google_enable=False, anonym=False, filtered=False, https=False):
    return FreeProxy(
        timeout=timeout, 
        google=google_enable, 
        anonym=anonym,
        elite=filtered,
        https=https,
        rand=True).get_proxy_list(repeat=True)

http_proxies = gen_proxy_list(timeout=5)

In [7]:
http_proxies

['5.78.124.240:40000',
 '150.136.247.129:1080',
 '13.38.176.104:3128',
 '180.210.89.215:3128',
 '44.218.183.55:80',
 '14.39.239.79:56270',
 '41.111.243.133:80',
 '45.87.68.6:15321',
 '3.127.121.101:80',
 '3.21.101.158:3128',
 '3.12.144.146:3128',
 '8.215.105.127:7777',
 '3.141.217.225:80',
 '3.90.100.12:80',
 '113.160.133.32:8080',
 '172.233.78.254:7890',
 '52.73.224.54:3128',
 '44.219.175.186:80',
 '158.160.52.208:8090',
 '85.215.64.49:80',
 '219.65.73.81:80',
 '18.223.25.15:80',
 '23.247.137.142:80',
 '67.43.236.19:30903',
 '50.232.104.86:80',
 '66.191.31.158:80',
 '37.187.25.85:80',
 '103.152.112.120:80',
 '188.40.59.208:3128',
 '23.247.136.245:80',
 '23.247.136.254:80',
 '103.152.112.157:80',
 '13.208.56.180:80',
 '35.72.118.126:80',
 '3.127.62.252:80',
 '43.201.121.81:80',
 '18.228.198.164:80',
 '52.67.10.183:80',
 '202.6.233.133:80',
 '43.154.134.238:50001',
 '3.129.184.210:80',
 '195.114.209.50:80',
 '51.16.199.206:3128',
 '13.48.109.48:3128',
 '51.17.58.162:3128',
 '3.97.176.25

In [ ]:
from tweeterpy import TweeterPy

try:
    twitter = TweeterPy(proxies = {'http': '222.252.194.204:8080'})
except Exception as e:
    if e == 'invalid response':
        print(f"Caught a generic exception: {e}")

In [ ]:
uid = twitter.get_user_id('omarsar0')

In [ ]:
uid

In [ ]:
user_data = twitter.get_user_data('omarsar0')

In [ ]:
user_data.keys()

In [ ]:
user_data.get('id')

In [ ]:
user_data.get('legacy_extended_profile')

In [ ]:
user_data

In [ ]:
user_data.get('legacy')

In [ ]:
from tweeterpy.util import User, Tweet

In [ ]:
user_info = User(user_data)

In [ ]:
tweet = twitter.get_tweet(tweet_id='1892223536493703501')

In [ ]:
tweet.get('data')['tweetResult']['result'].keys()

In [ ]:
tweet.get('data')['tweetResult']['result']['core']

In [ ]:
from tweeterpy import TweeterPy

try:
    twitter2 = TweeterPy(proxies = {'http': '85.215.64.49:80'})
except Exception as e:
    if e == 'invalid response':
        print(f"Caught a generic exception: {e}")

In [ ]:
tweet2 = twitter2.get_tweet(tweet_id='1892223536493703501')

In [ ]:
tweet2.get('api_rate_limit')

In [ ]:
y = twitter.get_user_data('elonmask')

In [8]:
import time
import datetime
from typing import Dict, List, Optional, Set

from tweeterpy import TweeterPy

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

MAX_RETRIES = 3
BACKOFF_FACTOR = 0.5
X_ALTER_URL = "https://xapi.betaco.tech/x-thread-api?url="

class TwitterKit:
    def __init__(
            self, 
            proxy_list, 
            x_login_name: Optional[str]=None,
            x_password: Optional[str]=None,
            x_login_email: Optional[str]=None,
            max_retires: Optional[int] = MAX_RETRIES
        ):
        """initiate twitter tools and set up parameters
        Args:
            proxy_lst: list of proxies in format like 'ip_addr:port'. support http proxies for now.
            x_login_name, x_password, x_login_email: X related login information. 
        Note:
            1. tweeterpy_client (based on tweeterpy package) is set up to get user id, user data and tweet given specific id.
            2. twikit_client (based on twikit package) is set up to get latest tweets from twitt account, retrieve tweets with replies.
            3. tweeterpy_client does not require login credentials, while twikit_client requires X related login information.
            4. tweeterpy_client is bound to rate limits constraint. It may resort to proxy to get over it.
            5. twikit_client use no proxy, since Twitter detects user's IP and may ban accounts with suspicious IP shifts.
            6. tweeterpy_clients_usage records client / proxy usage information for tweeterpy_client. It includes:
                - proxy: proxy used
                - initiate_tm: client first initiated
                - last_call_tm: client last called with API usage
                - remaining_requests: remaining usage cnt
                - next_reset_tm: rate limit next reset time
        """
        self.x_login_name = x_login_name
        self.x_password = x_password
        self.x_login_email = x_login_email
        self.max_retires = max_retires

        self.tweeterpy_clients_usage = [{'proxy': proxy} for proxy in proxy_list]  # save client / proxy usage information
        self.bad_proxies = set()  # store unusable proxies
        self.current_proxy = None

    def _load_tweeterpy_client(self, excluded_proxies: Optional[Set]=set()):
        """"load tweeterpy client
        Args:
            excluded_proxies: a set object with excluded proxies. (proxies connetable but may not work for specific function)
        """
        flag = 0

        # iterate all clients for usable one (both with connectable proxy and usage within rate limits)
        for idx, client_uage in enumerate(self.tweeterpy_clients_usage):
            if (client_uage.get('proxy') in excluded_proxies
                or client_uage.get('is_bad_proxy', False) == True   # client with bad proxy
                or (client_uage.get('remaining_requests', 20) <= 0 and client_uage.get('next_reset_tm') > int(time.time()))  # client restricted by rate limit
                ):
                continue
            else:
                try:
                    self.tweeterpy_client= TweeterPy(
                        proxies={'http': client_uage.get('proxy')}, 
                        log_level="INFO"
                    )
                    test_uid = self.tweeterpy_client.get_user_id('elonmask')  # test if client works
                    client_uage['initiate_tm'] = int(time.time())
                    self.current_proxy = client_uage['proxy']
                    flag = 1
                    break
                except Exception as e:
                    logging.warning(f"Failed to create TweeterPy client with proxy {client_uage['proxy']}: {e}")
                    client_uage['is_bad_proxy'] = True

        # no usable client
        if flag == 0:  
            logging.error(f"Exhausted all proxies and still could not establish TweeterPy client.")
            self.tweeterpy_client = None
            self.current_proxy = None


    def get_userid(self, username) -> str:
        """Get user ID based on user name (screen name like 'elonmusk')."""
        attempt = 0
        excluded_proxies = set()
        while attempt < self.max_retires:
            try:
                uid = self.tweeterpy_client.get_user_id(username)
                return uid
            except Exception as e:
                excluded_proxies.add(self.current_proxy)
                self._load_tweeterpy_client(excluded_proxies)
                attempt += 1
        return None


    def get_userdata(self, username):
        """get user profile based on user name (screen name like 'elonmusk')
        Args:
            username (str): user name (screen name like 'elonmusk')
        Usage:
            uid = user_data.get('rest_id')
            tweet_acct_info = user_data.get('legacy')
        """
        attempt = 0
        excluded_proxies = set()
        while attempt < self.max_retires:
            try:
                user_data = self.tweeterpy_client.get_user_data(username)
                return user_data
            except Exception as e:
                excluded_proxies.add(self.current_proxy)
                self._load_tweeterpy_client(excluded_proxies)
                attempt += 1
        return None


    def get_tweet_by_id(self, tweet_id):
        """Retrieves a tweet given specific tweet id.
        Args:
            username (str): user name (screen name like 'elonmusk')
            tweet_id (str): status id of tweet url
        Returns:
            tweet_dct (dict): information including tweet, user, and api usage
        Usage:
            tweet_id = tweet_dct.get('rest_id')  # tweet_id
            usage_data = tweet_dct.get('api_rate_limit')  # for api rate limit information
            tweet_info= tweet_dct.get('data', {}).get('tweetResult', {}).get('result', {})
            tweet_user_data = tweet_info.get('core', {}).get('user_results', {}).get('result', {})  # for user info
            tweet_data = tweet_info.get('legacy')  # for tweet info
        """
        attempt = 0
        excluded_proxies = set()
        while attempt < self.max_retires:
            try:
                tweet = self.tweeterpy_client.get_tweet(tweet_id)
                api_limit = tweet.get('api_rate_limit', {})
                # update client usage info
                idx = [x['proxy'] for x in self.tweeterpy_clients_usage].index(self.current_proxy)
                self.tweeterpy_clients_usage[idx]['last_call_tm'] = int(time.time())
                self.tweeterpy_clients_usage[idx]['remaining_requests'] = api_limit.get('remaining_requests_count')
                self.tweeterpy_clients_usage[idx]['next_reset_tm'] = int((datetime.datetime.now() + api_limit.get('reset_after_datetime_object')).timestamp())
                return tweet
            
            except Exception as e:
                excluded_proxies.add(self.current_proxy)
                self._load_tweeterpy_client(excluded_proxies)
                attempt += 1
        return None


    def get_tweets_by_user(self, username, total=20):
        """get user tweets based on user name (screen name like 'elonmusk').
           Not recommended since the tweets retrived are not arranged in time sequence.
        Args:
            username (str): user name (screen name like 'elonmusk')
        """
        attempt = 0
        excluded_proxies = set()
        while attempt < self.max_retires:
            try:
                user_tweets = self.tweeterpy_client.get_user_tweets(username, total=total)
                api_limit = user_tweets.get('api_rate_limit', {})
                # update client usage info
                idx = [x['proxy'] for x in self.tweeterpy_clients_usage].index(self.current_proxy)
                self.tweeterpy_clients_usage[idx]['last_call_tm'] = int(time.time())
                self.tweeterpy_clients_usage[idx]['remaining_requests'] = api_limit.get('remaining_requests_count')
                self.tweeterpy_clients_usage[idx]['next_reset_tm'] = int((datetime.datetime.now() + api_limit.get('reset_after_datetime_object')).timestamp())
                return user_tweets
            
            except Exception as e:
                excluded_proxies.add(self.current_proxy)
                self._load_tweeterpy_client()
                attempt += 1
        return None


In [9]:
tw_client_1 = TwitterKit(proxy_list=http_proxies)
tw_client_1._load_tweeterpy_client()
print('INITIATE')
print(f'current proxy: {tw_client_1.current_proxy}')
print(f"all client info: {tw_client_1.tweeterpy_clients_usage}")

screen_nm = 'omarsar0'
tweet_id = '1892223536493703501'

print('UID')
uid = tw_client_1.get_userid(screen_nm)
print(f'current proxy: {tw_client_1.current_proxy}')
print(f"all client info: {tw_client_1.tweeterpy_clients_usage}")

print('ACCT')
user_data = tw_client_1.get_userdata(screen_nm)
print(f'current proxy: {tw_client_1.current_proxy}')
print(f"all client info: {tw_client_1.tweeterpy_clients_usage}")

print('TWEET')
tweet_data = tw_client_1.get_tweet_by_id(tweet_id)
print(f'current proxy: {tw_client_1.current_proxy}')
print(f"all client info: {tw_client_1.tweeterpy_clients_usage}")
print(tweet_data)

print('USER_TWEET')
user_tweet_datra = tw_client_1.get_tweets_by_user(screen_nm)
print(f'current proxy: {tw_client_1.current_proxy}')
print(f"all client info: {tw_client_1.tweeterpy_clients_usage}")

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'twitter.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/minicon

2025-02-24 16:01:25,419 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-02-24 16:01:25,421 [INFO] :: API Updated Successfully.


/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


INITIATE
current proxy: 5.78.124.240:40000
all client info: [{'proxy': '5.78.124.240:40000', 'initiate_tm': 1740384085}, {'proxy': '150.136.247.129:1080'}, {'proxy': '13.38.176.104:3128'}, {'proxy': '180.210.89.215:3128'}, {'proxy': '44.218.183.55:80'}, {'proxy': '14.39.239.79:56270'}, {'proxy': '41.111.243.133:80'}, {'proxy': '45.87.68.6:15321'}, {'proxy': '3.127.121.101:80'}, {'proxy': '3.21.101.158:3128'}, {'proxy': '3.12.144.146:3128'}, {'proxy': '8.215.105.127:7777'}, {'proxy': '3.141.217.225:80'}, {'proxy': '3.90.100.12:80'}, {'proxy': '113.160.133.32:8080'}, {'proxy': '172.233.78.254:7890'}, {'proxy': '52.73.224.54:3128'}, {'proxy': '44.219.175.186:80'}, {'proxy': '158.160.52.208:8090'}, {'proxy': '85.215.64.49:80'}, {'proxy': '219.65.73.81:80'}, {'proxy': '18.223.25.15:80'}, {'proxy': '23.247.137.142:80'}, {'proxy': '67.43.236.19:30903'}, {'proxy': '50.232.104.86:80'}, {'proxy': '66.191.31.158:80'}, {'proxy': '37.187.25.85:80'}, {'proxy': '103.152.112.120:80'}, {'proxy': '188.4

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

current proxy: 5.78.124.240:40000
all client info: [{'proxy': '5.78.124.240:40000', 'initiate_tm': 1740384085}, {'proxy': '150.136.247.129:1080'}, {'proxy': '13.38.176.104:3128'}, {'proxy': '180.210.89.215:3128'}, {'proxy': '44.218.183.55:80'}, {'proxy': '14.39.239.79:56270'}, {'proxy': '41.111.243.133:80'}, {'proxy': '45.87.68.6:15321'}, {'proxy': '3.127.121.101:80'}, {'proxy': '3.21.101.158:3128'}, {'proxy': '3.12.144.146:3128'}, {'proxy': '8.215.105.127:7777'}, {'proxy': '3.141.217.225:80'}, {'proxy': '3.90.100.12:80'}, {'proxy': '113.160.133.32:8080'}, {'proxy': '172.233.78.254:7890'}, {'proxy': '52.73.224.54:3128'}, {'proxy': '44.219.175.186:80'}, {'proxy': '158.160.52.208:8090'}, {'proxy': '85.215.64.49:80'}, {'proxy': '219.65.73.81:80'}, {'proxy': '18.223.25.15:80'}, {'proxy': '23.247.137.142:80'}, {'proxy': '67.43.236.19:30903'}, {'proxy': '50.232.104.86:80'}, {'proxy': '66.191.31.158:80'}, {'proxy': '37.187.25.85:80'}, {'proxy': '103.152.112.120:80'}, {'proxy': '188.40.59.208:

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


current proxy: 5.78.124.240:40000
all client info: [{'proxy': '5.78.124.240:40000', 'initiate_tm': 1740384085, 'last_call_tm': 1740384086, 'remaining_requests': 49, 'next_reset_tm': 1740384986}, {'proxy': '150.136.247.129:1080'}, {'proxy': '13.38.176.104:3128'}, {'proxy': '180.210.89.215:3128'}, {'proxy': '44.218.183.55:80'}, {'proxy': '14.39.239.79:56270'}, {'proxy': '41.111.243.133:80'}, {'proxy': '45.87.68.6:15321'}, {'proxy': '3.127.121.101:80'}, {'proxy': '3.21.101.158:3128'}, {'proxy': '3.12.144.146:3128'}, {'proxy': '8.215.105.127:7777'}, {'proxy': '3.141.217.225:80'}, {'proxy': '3.90.100.12:80'}, {'proxy': '113.160.133.32:8080'}, {'proxy': '172.233.78.254:7890'}, {'proxy': '52.73.224.54:3128'}, {'proxy': '44.219.175.186:80'}, {'proxy': '158.160.52.208:8090'}, {'proxy': '85.215.64.49:80'}, {'proxy': '219.65.73.81:80'}, {'proxy': '18.223.25.15:80'}, {'proxy': '23.247.137.142:80'}, {'proxy': '67.43.236.19:30903'}, {'proxy': '50.232.104.86:80'}, {'proxy': '66.191.31.158:80'}, {'pro

/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


current proxy: 5.78.124.240:40000
all client info: [{'proxy': '5.78.124.240:40000', 'initiate_tm': 1740384085, 'last_call_tm': 1740384088, 'remaining_requests': 49, 'next_reset_tm': 1740384987}, {'proxy': '150.136.247.129:1080'}, {'proxy': '13.38.176.104:3128'}, {'proxy': '180.210.89.215:3128'}, {'proxy': '44.218.183.55:80'}, {'proxy': '14.39.239.79:56270'}, {'proxy': '41.111.243.133:80'}, {'proxy': '45.87.68.6:15321'}, {'proxy': '3.127.121.101:80'}, {'proxy': '3.21.101.158:3128'}, {'proxy': '3.12.144.146:3128'}, {'proxy': '8.215.105.127:7777'}, {'proxy': '3.141.217.225:80'}, {'proxy': '3.90.100.12:80'}, {'proxy': '113.160.133.32:8080'}, {'proxy': '172.233.78.254:7890'}, {'proxy': '52.73.224.54:3128'}, {'proxy': '44.219.175.186:80'}, {'proxy': '158.160.52.208:8090'}, {'proxy': '85.215.64.49:80'}, {'proxy': '219.65.73.81:80'}, {'proxy': '18.223.25.15:80'}, {'proxy': '23.247.137.142:80'}, {'proxy': '67.43.236.19:30903'}, {'proxy': '50.232.104.86:80'}, {'proxy': '66.191.31.158:80'}, {'pro

## Twikit

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
import asyncio
from twikit import Client

USERNAME = 'ai4fun2004'
EMAIL = 'ai4fun2004@gmail.com'
PASSWORD = 'ai4fun_test'

# Initialize client
client = Client('en-US')

await client.login(
        auth_info_1=USERNAME,
        auth_info_2=EMAIL,
        password=PASSWORD,
        cookies_file='cookies.json'
    )

In [114]:
async def async_get_user_info(self, screen_nm, uid):
    attempt = 0
    if screen_nm is None and uid is None:
        return None
    
    while attempt < self.max_retires:
        try:
            if screen_nm is not None:
                return await client.get_user_by_screen_name(screen_nm)
            elif uid is not None:
                return await client.get_user_by_screen_name(uid)
        except:
            attempt += 1
    return None


In [54]:
user_data

{'__typename': 'User',
 'id': 'VXNlcjozNDQ4Mjg0MzEz',
 'rest_id': '3448284313',
 'affiliates_highlighted_label': {},
 'parody_commentary_fan_label': 'None',
 'is_blue_verified': True,
 'profile_image_shape': 'Circle',
 'legacy': {'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
  'entities': {'description': {'urls': []},
   'url': {'urls': [{'display_url': 'dair-ai.thinkific.com',
      'expanded_url': 'https://dair-ai.thinkific.com/',
      'url': 'https://t.co/JBU5beHQNs',
      'indices': [0, 23]}]}},
  'fast_followers_count': 0,
  'favourites_count': 29049,
  'followers_count': 231264,
  'friends_count': 576,
  'has_custom_timelines': True,
  'is_translator': False,
  'listed_count': 3891,
  'location': '',
  'media_count': 2951,
  'name': 'elvi

In [57]:
user_data.keys()

dict_keys(['__typename', 'id', 'rest_id', 'affiliates_highlighted_label', 'parody_commentary_fan_label', 'is_blue_verified', 'profile_image_shape', 'legacy', 'professional', 'tipjar_settings', 'legacy_extended_profile', 'is_profile_translatable', 'has_hidden_subscriptions_on_profile', 'verification_info', 'highlights_info', 'user_seed_tweet_count', 'business_account', 'creator_subscriptions_count'])

In [58]:
user_data.get('legacy', {}).get('entities', {}).get('url', {}).get('urls')

[{'display_url': 'dair-ai.thinkific.com',
  'expanded_url': 'https://dair-ai.thinkific.com/',
  'url': 'https://t.co/JBU5beHQNs',
  'indices': [0, 23]}]

In [55]:
user_data.get('rest_id')

'3448284313'

In [59]:
user_data.get('legacy', {}).get('entities', {}).get('description', {}).get('urls')

[]

In [26]:
user_data['is_blue_verified']

True

In [16]:
common_keys = user_data['legacy'].keys() & user.__dict__.keys()

In [20]:
for key in user_data['legacy'].keys():
    if key not in list(common_keys):
        print(f"key:{key} value:{user_data['legacy'][key]}")

key:entities value:{'description': {'urls': []}, 'url': {'urls': [{'display_url': 'dair-ai.thinkific.com', 'expanded_url': 'https://dair-ai.thinkific.com/', 'url': 'https://t.co/JBU5beHQNs', 'indices': [0, 23]}]}}
key:friends_count value:576
key:pinned_tweet_ids_str value:['1893394584903766313']
key:profile_image_url_https value:https://pbs.twimg.com/profile_images/939313677647282181/vZjFWtAn_normal.jpg
key:profile_interstitial_type value:


In [ ]:
profile_image_url

In [21]:
for key in user.__dict__.keys():
    if key not in list(common_keys):
        print(f"key:{key} value:{user.__dict__[key]}")

key:_client value:<twikit.client.client.Client object at 0x7c677ddc5f90>
key:id value:3448284313
key:profile_image_url value:https://pbs.twimg.com/profile_images/939313677647282181/vZjFWtAn_normal.jpg
key:description_urls value:[]
key:urls value:[{'display_url': 'dair-ai.thinkific.com', 'expanded_url': 'https://dair-ai.thinkific.com/', 'url': 'https://t.co/JBU5beHQNs', 'indices': [0, 23]}]
key:pinned_tweet_ids value:['1893394584903766313']
key:is_blue_verified value:True
key:can_dm value:True
key:can_media_tag value:True
key:want_retweets value:False
key:following_count value:576
key:protected value:False


In [15]:
user_data['legacy']

{'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'dair-ai.thinkific.com',
     'expanded_url': 'https://dair-ai.thinkific.com/',
     'url': 'https://t.co/JBU5beHQNs',
     'indices': [0, 23]}]}},
 'fast_followers_count': 0,
 'favourites_count': 29049,
 'followers_count': 231264,
 'friends_count': 576,
 'has_custom_timelines': True,
 'is_translator': False,
 'listed_count': 3891,
 'location': '',
 'media_count': 2951,
 'name': 'elvis',
 'normal_followers_count': 231264,
 'pinned_tweet_ids_str': ['1893394584903766313'],
 'possibly_sensitive': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3448284313/1565974901',
 'profile_image_url_https': 'https://pbs.t

In [5]:
user.__dict__

{'_client': <twikit.client.client.Client at 0x7c677ddc5f90>,
 'id': '3448284313',
 'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
 'name': 'elvis',
 'screen_name': 'omarsar0',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/939313677647282181/vZjFWtAn_normal.jpg',
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3448284313/1565974901',
 'url': 'https://t.co/JBU5beHQNs',
 'location': '',
 'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
 'description_urls': [],
 'urls': [{'display_url': 'dair-ai.thinkific.com',
   'expanded_url': 'https://dair-ai.thinkific.com/',
   'url': 'https://t.co/JBU5beHQNs',
   'indices': [0, 23]}],
 'pinned_tweet_ids': ['1893394584903766313'],
 'is_blue_verified': True,
 'verified': False,
 'possibly_sensitive': False,
 'can_dm': True,
 'can_media_tag': True,
 'want_retweets': False,
 'default_profile': False

In [10]:
user.id

'3448284313'

In [11]:
user2 = await client.get_user_by_id('3448284313')

2025-02-24 16:04:21,745 [INFO] :: HTTP Request: GET https://x.com/i/api/graphql/tD8zKvQzwY3kdx5yz6YmOw/UserByRestId?variables=%7B%22userId%22%3A+%223448284313%22%2C+%22withSafetyModeUserFields%22%3A+true%7D&features=%7B%22hidden_profile_likes_enabled%22%3A+true%2C+%22hidden_profile_subscriptions_enabled%22%3A+true%2C+%22responsive_web_graphql_exclude_directive_enabled%22%3A+true%2C+%22verified_phone_label_enabled%22%3A+false%2C+%22subscriptions_verification_info_is_identity_verified_enabled%22%3A+true%2C+%22subscriptions_verification_info_verified_since_enabled%22%3A+true%2C+%22highlights_tweets_tab_ui_enabled%22%3A+true%2C+%22responsive_web_twitter_article_notes_tab_enabled%22%3A+false%2C+%22creator_subscriptions_tweet_preview_api_enabled%22%3A+true%2C+%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3A+false%2C+%22responsive_web_graphql_timeline_navigation_enabled%22%3A+true%7D "HTTP/1.1 200 OK"


In [13]:
user2.__dict__

{'_client': <twikit.client.client.Client at 0x7c677ddc5f90>,
 'id': '3448284313',
 'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
 'name': 'elvis',
 'screen_name': 'omarsar0',
 'profile_image_url': 'https://pbs.twimg.com/profile_images/939313677647282181/vZjFWtAn_normal.jpg',
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3448284313/1565974901',
 'url': 'https://t.co/JBU5beHQNs',
 'location': '',
 'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
 'description_urls': [],
 'urls': [{'display_url': 'dair-ai.thinkific.com',
   'expanded_url': 'https://dair-ai.thinkific.com/',
   'url': 'https://t.co/JBU5beHQNs',
   'indices': [0, 23]}],
 'pinned_tweet_ids': ['1893394584903766313'],
 'is_blue_verified': True,
 'verified': False,
 'possibly_sensitive': False,
 'can_dm': True,
 'can_media_tag': True,
 'want_retweets': False,
 'default_profile': False

In [12]:
user2

<User id="3448284313">

In [ ]:
user = await client.get_user_by_screen_name('omarsar0')

In [ ]:
async def async_search_tweets( 
    query: str,
    product: Literal['Top', 'Latest', 'Media'],
    count: int = 20,
    cursor: str | None = None
) -> Coroutine[Any, Any, Result[Tweet]]):
    attempt = 0
    
    while attempt < self.max_retires:
        try:
            return client.search_tweet('llm reasoning', 'Latest')
        except:
            attempt += 1
    return None

In [70]:
tweet_data['data']['tweetResult']['result'].keys()

dict_keys(['__typename', 'rest_id', 'core', 'unmention_data', 'edit_control', 'is_translatable', 'views', 'source', 'grok_analysis_button', 'legacy'])

In [63]:
tweet_data['data']['tweetResult']['result'].keys()

dict_keys(['__typename', 'rest_id', 'core', 'unmention_data', 'edit_control', 'is_translatable', 'views', 'source', 'grok_analysis_button', 'legacy'])

In [77]:
tweet_data['data']['tweetResult']['result']['views']

{'count': '5888', 'state': 'EnabledWithCount'}

In [83]:
tweet_data['data']['tweetResult']['result']['core']['user_results']['result'].keys()

dict_keys(['__typename', 'id', 'rest_id', 'affiliates_highlighted_label', 'parody_commentary_fan_label', 'is_blue_verified', 'profile_image_shape', 'legacy', 'professional', 'tipjar_settings'])

In [ ]:
acct_data = tweet_data['data']['tweetResult']['result']['core']

{'user_results': {'result': {'__typename': 'User',
   'id': 'VXNlcjozNDQ4Mjg0MzEz',
   'rest_id': '3448284313',
   'affiliates_highlighted_label': {},
   'parody_commentary_fan_label': 'None',
   'is_blue_verified': True,
   'profile_image_shape': 'Circle',
   'legacy': {'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
    'default_profile': False,
    'default_profile_image': False,
    'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
    'entities': {'description': {'urls': []},
     'url': {'urls': [{'display_url': 'dair-ai.thinkific.com',
        'expanded_url': 'https://dair-ai.thinkific.com/',
        'url': 'https://t.co/JBU5beHQNs',
        'indices': [0, 23]}]}},
    'fast_followers_count': 0,
    'favourites_count': 29049,
    'followers_count': 231264,
    'friends_count': 576,
    'has_custom_timelines': True,
    'is_translator': False,
    'liste

In [96]:
tweet_data_1 = tweet_data['data']['tweetResult']['result']['legacy']

In [ ]:
tweet.__dict__.keys()

dict_keys(['_client', '_data', '_legacy', 'user', 'replies', 'reply_to', 'related_tweets', 'thread'])

In [95]:
tweet.__dict__['_legacy']

{'bookmark_count': 89,
 'bookmarked': False,
 'created_at': 'Fri Feb 21 14:32:02 +0000 2025',
 'conversation_id_str': '1892945381174210933',
 'display_text_range': [0, 279],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.x.com/uCjUp1JVSe',
    'expanded_url': 'https://x.com/omarsar0/status/1892945381174210933/photo/1',
    'id_str': '1892945377290047489',
    'indices': [280, 303],
    'media_key': '3_1892945377290047489',
    'media_url_https': 'https://pbs.twimg.com/media/GkUXypiXUAE7TNX.png',
    'type': 'photo',
    'url': 'https://t.co/uCjUp1JVSe',
    'ext_media_availability': {'status': 'Available'},
    'features': {'large': {'faces': []},
     'medium': {'faces': []},
     'small': {'faces': []},
     'orig': {'faces': []}},
    'sizes': {'large': {'h': 843, 'w': 775, 'resize': 'fit'},
     'medium': {'h': 843, 'w': 775, 'resize': 'fit'},
     'small': {'h': 680, 'w': 625, 'resize': 'fit'},
     'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}},
    'original_in

In [ ]:
tweet.__dict__['_legacy']

In [ ]:
tweet_data['data']['tweetResult']['result']

In [ ]:
tweet.__dict__['_data'].keys()

dict_keys(['__typename', 'rest_id', 'core', 'unmention_data', 'edit_control', 'is_translatable', 'views', 'source', 'note_tweet', 'legacy', 'quick_promote_eligibility'])

In [111]:
tweet_id = "1893737780246700386"

In [112]:
tweet_info = tw_client_1.get_tweet_by_id(tweet_id)
tweet_data_1 = tweet_info['data']['tweetResult']['result']['legacy']


/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2025-02-24 18:00:22,004 [ERROR] :: Error code 239 - Bad guest token

Traceback (most recent call last):
  File "/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/tweeterpy/utils/request.py", line 40, in request
    return util.check_for_errors(response)
  File "/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/tweeterpy/util.py", line 139, in check_for_errors
    raise Exception(error_message)
Exception: Error code 239 - Bad guest token


/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'twitter.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/minicon

2025-02-24 18:00:28,876 [WARNING] :: Skipping API Updates. Couldn't get the latest API data.
2025-02-24 18:00:28,879 [INFO] :: API Updated Successfully.


/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/jiezi/miniconda3/envs/ai4fun/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'x.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
tweet_data_1

In [115]:
tweet_info2 = await client.get_tweet_by_id(tweet_id)
tweet_data_2 = tweet_info2.__dict__['_data']['legacy']

2025-02-24 18:01:58,469 [INFO] :: HTTP Request: GET https://x.com/i/api/graphql/U0HTv-bAWTBYylwEMT7x5A/TweetDetail?variables=%7B%22focalTweetId%22%3A+%221893737780246700386%22%2C+%22with_rux_injections%22%3A+false%2C+%22includePromotedContent%22%3A+true%2C+%22withCommunity%22%3A+true%2C+%22withQuickPromoteEligibilityTweetFields%22%3A+true%2C+%22withBirdwatchNotes%22%3A+true%2C+%22withVoice%22%3A+true%2C+%22withV2Timeline%22%3A+true%7D&features=%7B%22creator_subscriptions_tweet_preview_api_enabled%22%3A+true%2C+%22c9s_tweet_anatomy_moderator_badge_enabled%22%3A+true%2C+%22tweetypie_unmention_optimization_enabled%22%3A+true%2C+%22responsive_web_edit_tweet_api_enabled%22%3A+true%2C+%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3A+true%2C+%22view_counts_everywhere_api_enabled%22%3A+true%2C+%22longform_notetweets_consumption_enabled%22%3A+true%2C+%22responsive_web_twitter_article_tweet_consumption_enabled%22%3A+true%2C+%22tweet_awards_web_tipping_enabled%22%3A+false%2C+%2

In [117]:
tweet_info2.__dict__.keys()

dict_keys(['_client', '_data', '_legacy', 'user', 'replies', 'reply_to', 'related_tweets', 'thread'])

In [127]:
tweet_info2.__dict__['reply_to']

[<Tweet id="1893736193432359239">]

In [124]:
tweet_info2.__dict__['reply_to'][0].__dict__

{'_client': <twikit.client.client.Client at 0x7c677ddc5f90>,
 '_data': {'__typename': 'Tweet',
  'rest_id': '1893736193432359239',
  'has_birdwatch_notes': False,
  'core': {'user_results': {'result': {'__typename': 'User',
     'id': 'VXNlcjoxODY0MjA1NTE=',
     'rest_id': '186420551',
     'affiliates_highlighted_label': {},
     'has_graduated_access': True,
     'is_blue_verified': True,
     'profile_image_shape': 'Circle',
     'legacy': {'can_dm': True,
      'can_media_tag': True,
      'created_at': 'Fri Sep 03 12:25:12 +0000 2010',
      'default_profile': False,
      'default_profile_image': False,
      'description': 'Co-founder & CEO @HuggingFace 🤗, the open and collaborative platform for AI builders',
      'entities': {'description': {'urls': []},
       'url': {'urls': [{'display_url': 'huggingface.co/clem',
          'expanded_url': 'http://huggingface.co/clem',
          'url': 'https://t.co/7OMlqfI2Rh',
          'indices': [0, 23]}]}},
      'fast_followers_count'

In [ ]:
reply_to = tweet_info2.__dict__['reply_to'][0]
in_reply_to_status_id_str = tweet_info2.__dict__['reply_to'][0].__dict__['_data']['rest_id']
in_reply_to_user_id_str = tweet_info2.__dict__['reply_to'][0].__dict__['_data']['core']['rest_id']
in_reply_to_screen_name = tweet_info2.__dict__['reply_to'][0].__dict__['_data']['core']['user_results']['result']['legacy']['screen_name']

dict_keys(['_client', '_data', '_legacy', 'user', 'replies', 'reply_to', 'related_tweets', 'thread'])

In [ ]:
tweet_info2.__dict__['reply_to'][0].__dict__['_data']['core']['user_results']['result']['legacy']['screen_name']

{'__typename': 'Tweet',
 'rest_id': '1893736193432359239',
 'has_birdwatch_notes': False,
 'core': {'user_results': {'result': {'__typename': 'User',
    'id': 'VXNlcjoxODY0MjA1NTE=',
    'rest_id': '186420551',
    'affiliates_highlighted_label': {},
    'has_graduated_access': True,
    'is_blue_verified': True,
    'profile_image_shape': 'Circle',
    'legacy': {'can_dm': True,
     'can_media_tag': True,
     'created_at': 'Fri Sep 03 12:25:12 +0000 2010',
     'default_profile': False,
     'default_profile_image': False,
     'description': 'Co-founder & CEO @HuggingFace 🤗, the open and collaborative platform for AI builders',
     'entities': {'description': {'urls': []},
      'url': {'urls': [{'display_url': 'huggingface.co/clem',
         'expanded_url': 'http://huggingface.co/clem',
         'url': 'https://t.co/7OMlqfI2Rh',
         'indices': [0, 23]}]}},
     'fast_followers_count': 0,
     'favourites_count': 7834,
     'followers_count': 127606,
     'friends_count': 46

In [ ]:
in_reply_to_status_id_str = tweet_info2.__dict__['reply_to'][0].__dict__['_data']['rest_id']

In [ ]:
tweet_info2.__dict__['reply_to'][0]['']

In [ ]:
tweet_data_2 = tweet.__dict__['_data']['legacy']

In [98]:
tweet_data_1.keys()

dict_keys(['bookmark_count', 'bookmarked', 'created_at', 'conversation_id_str', 'display_text_range', 'entities', 'extended_entities', 'favorite_count', 'favorited', 'full_text', 'in_reply_to_screen_name', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'possibly_sensitive', 'possibly_sensitive_editable', 'quote_count', 'reply_count', 'retweet_count', 'retweeted', 'user_id_str', 'id_str'])

In [99]:
tweet_data_2.keys()

dict_keys(['bookmark_count', 'bookmarked', 'created_at', 'conversation_id_str', 'display_text_range', 'entities', 'extended_entities', 'favorite_count', 'favorited', 'full_text', 'is_quote_status', 'lang', 'possibly_sensitive', 'possibly_sensitive_editable', 'quote_count', 'reply_count', 'retweet_count', 'retweeted', 'user_id_str', 'id_str'])

In [103]:
common_keys = tweet_data_1.keys() & tweet_data_2.keys()

In [120]:
tweet_data_1['in_reply_to_screen_name']

'ClementDelangue'

In [121]:
tweet_data_1['in_reply_to_status_id_str']

'1893736193432359239'

In [122]:
tweet_data_1['in_reply_to_user_id_str']

'186420551'

In [106]:
for key in tweet_data_1.keys():
    if key not in common_keys:
        print(key)

in_reply_to_screen_name
in_reply_to_status_id_str
in_reply_to_user_id_str


In [107]:
for key in tweet_data_2.keys():
    if key not in common_keys:
        print(key)

In [110]:
tweet.__dict__['replies']

[<Tweet id="1892945394096828627">]

In [108]:
tweet.__dict__.keys()

dict_keys(['_client', '_data', '_legacy', 'user', 'replies', 'reply_to', 'related_tweets', 'thread'])

In [89]:
tweet.__dict__['_data']['core']['user_results']['result'].keys()

dict_keys(['__typename', 'id', 'rest_id', 'affiliates_highlighted_label', 'has_graduated_access', 'is_blue_verified', 'profile_image_shape', 'legacy', 'professional'])

In [90]:
tweet.__dict__['_data']['core']['user_results']['result']['legacy']

{'can_dm': True,
 'can_media_tag': True,
 'created_at': 'Fri Sep 04 12:59:26 +0000 2015',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'Building with AI agents @dair_ai • Prev: Meta AI, Galactica LLM, Elastic, PaperswithCode, PhD • I also teach how to leverage and build with LLMs & AI Agents ⬇️',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'dair-ai.thinkific.com',
     'expanded_url': 'https://dair-ai.thinkific.com/',
     'url': 'https://t.co/JBU5beHQNs',
     'indices': [0, 23]}]}},
 'fast_followers_count': 0,
 'favourites_count': 29049,
 'followers_count': 231272,
 'friends_count': 576,
 'has_custom_timelines': True,
 'is_translator': False,
 'listed_count': 3891,
 'location': '',
 'media_count': 2951,
 'name': 'elvis',
 'normal_followers_count': 231272,
 'pinned_tweet_ids_str': ['1893394584903766313'],
 'possibly_sensitive': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3448284313/1565974901',
 

In [67]:
tweet.__dict__['_data']['quick_promote_eligibility']

{'eligibility': 'IneligibleNotProfessional'}

In [52]:
str(tweet.__dict__['_client'])

'<twikit.client.client.Client object at 0x7c677ddc5f90>'

In [35]:
tweet.__dict__

{'_client': <twikit.client.client.Client at 0x7c677ddc5f90>,
 '_data': {'__typename': 'Tweet',
  'rest_id': '1893803151725084902',
  'core': {'user_results': {'result': {'__typename': 'User',
     'id': 'VXNlcjoxMDEwNjk5Mg==',
     'rest_id': '10106992',
     'affiliates_highlighted_label': {},
     'has_graduated_access': True,
     'is_blue_verified': True,
     'profile_image_shape': 'Circle',
     'legacy': {'can_dm': True,
      'can_media_tag': True,
      'created_at': 'Fri Nov 09 19:05:15 +0000 2007',
      'default_profile': False,
      'default_profile_image': False,
      'description': 'Information enthusiast 📚, Portlander 🌲, engineering manager @SlackHQ working on infrastructure and distributed databases (formerly: @newrelic, @surescripts)',
      'entities': {'description': {'urls': []},
       'url': {'urls': [{'display_url': 'martymatheny.com',
          'expanded_url': 'http://martymatheny.com',
          'url': 'https://t.co/jLyHg46tJB',
          'indices': [0, 23]}

In [34]:
tweet.related_tweets

In [27]:
tweets = await client.search_tweet('llm reasoning', 'Latest')

for tweet in tweets:
    print(
        tweet.user.name,
        tweet.text,
        tweet.created_at
    )

2025-02-24 16:31:36,705 [INFO] :: HTTP Request: GET https://x.com/i/api/graphql/flaR-PUMshxFWZWPNpq4zA/SearchTimeline?variables=%7B%22rawQuery%22%3A+%22llm+reasoning%22%2C+%22count%22%3A+20%2C+%22querySource%22%3A+%22typed_query%22%2C+%22product%22%3A+%22Latest%22%7D&features=%7B%22creator_subscriptions_tweet_preview_api_enabled%22%3A+true%2C+%22c9s_tweet_anatomy_moderator_badge_enabled%22%3A+true%2C+%22tweetypie_unmention_optimization_enabled%22%3A+true%2C+%22responsive_web_edit_tweet_api_enabled%22%3A+true%2C+%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3A+true%2C+%22view_counts_everywhere_api_enabled%22%3A+true%2C+%22longform_notetweets_consumption_enabled%22%3A+true%2C+%22responsive_web_twitter_article_tweet_consumption_enabled%22%3A+true%2C+%22tweet_awards_web_tipping_enabled%22%3A+false%2C+%22longform_notetweets_rich_text_read_enabled%22%3A+true%2C+%22longform_notetweets_inline_media_enabled%22%3A+true%2C+%22rweb_video_timestamps_enabled%22%3A+true%2C+%22respo

In [ ]:
async def async_get_user_tweets( 
    user_id: str,
    tweet_type: Literal['Tweets', 'Replies', 'Media', 'Likes'],
    count: int = 40,
    cursor: str | None = None
) -> Coroutine[Any, Any, Result[Tweet]]:
    attempt = 0
    
    while attempt < self.max_retires:
        try:
            return client.get_user_tweets(user.id, 'Tweets')
        except:
            attempt += 1
    return None

In [39]:
tweet.__dict__.keys()

dict_keys(['_client', '_data', '_legacy', 'user', 'replies', 'reply_to', 'related_tweets', 'thread'])

In [ ]:
tweet.__dict__['_legacy']['created_at']

{'bookmark_count': 89,
 'bookmarked': False,
 'created_at': 'Fri Feb 21 14:32:02 +0000 2025',
 'conversation_id_str': '1892945381174210933',
 'display_text_range': [0, 279],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.x.com/uCjUp1JVSe',
    'expanded_url': 'https://x.com/omarsar0/status/1892945381174210933/photo/1',
    'id_str': '1892945377290047489',
    'indices': [280, 303],
    'media_key': '3_1892945377290047489',
    'media_url_https': 'https://pbs.twimg.com/media/GkUXypiXUAE7TNX.png',
    'type': 'photo',
    'url': 'https://t.co/uCjUp1JVSe',
    'ext_media_availability': {'status': 'Available'},
    'features': {'large': {'faces': []},
     'medium': {'faces': []},
     'small': {'faces': []},
     'orig': {'faces': []}},
    'sizes': {'large': {'h': 843, 'w': 775, 'resize': 'fit'},
     'medium': {'h': 843, 'w': 775, 'resize': 'fit'},
     'small': {'h': 680, 'w': 625, 'resize': 'fit'},
     'thumb': {'h': 150, 'w': 150, 'resize': 'crop'}},
    'original_in

In [ ]:
['core']['user_results']['result'].keys()

In [46]:
for tweet in tweets:
    print(tweet.text, tweet.__dict__['_legacy']['created_at'])

RT @omarsar0: This is extremely handy!

Chat with any AI with this all-in-one CLI tool.

The CMD mode works really well. 

Supports differe… Sun Feb 23 16:44:13 +0000 2025
RT @dair_ai: Here are the top AI Papers of the Week (Feb 10-16):   

- AI Co-Scientist
- Open-Reasoner-Zero
- The AI CUDA Engineer
- Native… Sun Feb 23 16:24:56 +0000 2025
RT @omarsar0: openai's deep research always asks for clarification

it's a really important feature

ai coding agents like windsurf and cur… Sun Feb 23 03:41:51 +0000 2025
This is extremely handy!

Chat with any AI with this all-in-one CLI tool.

The CMD mode works really well. 

Supports different models, modes, input types, session handling, RAG use cases, function calling, and more. https://t.co/OUNIa9j6ry Sat Feb 22 21:23:35 +0000 2025
@windsurf_ai any thoughts? Sat Feb 22 20:40:56 +0000 2025
openai's deep research always asks for clarification

it's a really important feature

ai coding agents like windsurf and cursor can also benefit from tha

In [ ]:
tweets = await client.get_user_tweets(user.id, 'Tweets')

tweets_info = []
for tweet in tweets:
    tweets_info.append(tweet)
    print(tweet.text, tweet.__dict__['_legacy']['created_at'])

2025-02-24 16:45:28,956 [INFO] :: HTTP Request: GET https://x.com/i/api/graphql/QWF3SzpHmykQHsQMixG0cg/UserTweets?variables=%7B%22userId%22%3A+%223448284313%22%2C+%22count%22%3A+40%2C+%22includePromotedContent%22%3A+true%2C+%22withQuickPromoteEligibilityTweetFields%22%3A+true%2C+%22withVoice%22%3A+true%2C+%22withV2Timeline%22%3A+true%7D&features=%7B%22creator_subscriptions_tweet_preview_api_enabled%22%3A+true%2C+%22c9s_tweet_anatomy_moderator_badge_enabled%22%3A+true%2C+%22tweetypie_unmention_optimization_enabled%22%3A+true%2C+%22responsive_web_edit_tweet_api_enabled%22%3A+true%2C+%22graphql_is_translatable_rweb_tweet_is_translatable_enabled%22%3A+true%2C+%22view_counts_everywhere_api_enabled%22%3A+true%2C+%22longform_notetweets_consumption_enabled%22%3A+true%2C+%22responsive_web_twitter_article_tweet_consumption_enabled%22%3A+true%2C+%22tweet_awards_web_tipping_enabled%22%3A+false%2C+%22longform_notetweets_rich_text_read_enabled%22%3A+true%2C+%22longform_notetweets_inline_media_enable

In [47]:
trends = await client.get_trends('trending')

2025-02-24 16:50:20,973 [INFO] :: HTTP Request: GET https://x.com/i/api/2/guide.json?count=20&include_page_configuration=true&initial_tab_id=trending "HTTP/1.1 200 OK"


In [49]:
trends[0].__dict__

{'_client': <twikit.client.client.Client at 0x7c677ddc5f90>,
 'name': '#NewWingstopTenders',
 'tweets_count': 'Promoted by Wingstop',
 'domain_context': None,
 'grouped_trends': []}

In [50]:
trends[1].__dict__

{'_client': <twikit.client.client.Client at 0x7c677ddc5f90>,
 'name': 'Dan Bongino',
 'tweets_count': None,
 'domain_context': 'Trending',
 'grouped_trends': ['Deputy Director of the FBI', 'Kash and Dan']}

In [48]:
trends

[<Trend name="#NewWingstopTenders">,
 <Trend name="Dan Bongino">,
 <Trend name="Tommie">,
 <Trend name="taemin">,
 <Trend name="#OVERTURE_Daytime">,
 <Trend name="George Harris">,
 <Trend name="#TheWhiteLotus">,
 <Trend name="Congrats Dan">,
 <Trend name="Sean Perry">,
 <Trend name="#STARDOM">,
 <Trend name="Shai">,
 <Trend name="Jane Fonda">,
 <Trend name="#SAGAwards">,
 <Trend name="#Married2Med">,
 <Trend name="Chet">,
 <Trend name="Timmy">,
 <Trend name="Quad">,
 <Trend name="Secret Service">,
 <Trend name="Xenofobia">,
 <Trend name="Cutesy">,
 <Trend name="Jaylen Clark">,
 <Trend name="Hartenstein">,
 <Trend name="Carville">,
 <Trend name="Traditionally">,
 <Trend name="Fusion Ken">,
 <Trend name="Ty Jerome">,
 <Trend name="Naz Reid">,
 <Trend name="Emilia Perez">,
 <Trend name="Megyn">,
 <Trend name="Venezolano">]

In [ ]:
Retrieves the timeline. Retrieves tweets from Home -> For You.
tweets = await client.get_timeline()
for tweet in tweets:
    print(tweet)
<Tweet id="...">
<Tweet id="...">
...
...
more_tweets = await tweets.next() # Retrieve more tweets
for tweet in more_tweets:
    print(tweet)
<Tweet id="...">
<Tweet id="...">

In [ ]:
Retrieves the timeline. Retrieves tweets from Home -> Following.
tweets = await client.get_latest_timeline()
for tweet in tweets:
    print(tweet)
<Tweet id="...">
<Tweet id="...">
...
...
more_tweets = await tweets.next() # Retrieve more tweets
for tweet in more_tweets:
    print(tweet)
<Tweet id="...">
<Tweet id="...">
...
...

In [ ]:
import asyncio
from crawl4ai import *

async def main():
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://xcancel.com/omarsar0",
        )
        print(result.markdown)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
export X_USERNAME='ai4fun2004'
export X_PASSWORD='ai4fun_test'
export X_EMAIL='ai4fun2004@gmail.com'


In [ ]:
user.id

In [ ]:
tweets = await client.get_user_tweets(user.id, 'Tweets')

tweets_info = []
for tweet in tweets:
    tweets_info.append(tweet)
    print(tweet.text)

In [ ]:
len(tweets_info)

In [ ]:
tweets_info[0]

In [ ]:
help(tweets_info[0])